# Recommender Systems

### Learning Objective

- Learn what is a recommender system and it's business applications
- Learn what kind of data is required for building recommendation systems
- Learn recommendation techniques like associtation rules and collaborative filtering
- Learn how to build, evaluate recommendation systems using python libraries

### Overview

- Recommendation systems are simple algorithms which recommends movies, music, news, books, articles, groceries, hotels or vacation destinations and act as backbone for cross selling across industries. 
- It leverages customer's behavioral data for recommending new products or services. 

Examples of recommenation systems that have been deployed in the industry:

- Amazon's "Customers who buys this item also bought"
- Netflix's "shows and movies you may want to watch". 


In this session, we will discuss two key algorithms that are widely used for building these recommendation systems. They are:

- Association Rules
- Collaborative Filtering

### Association Rules

- Association rule finds combinations of items that frequently occur together in orders or baskets. 

- The items that frequently occur together are called itemsets. 

- Itemsets help discover relationships between items that people buy together and use that as a basis for creating strategies like combining products as combo offer or place products next to each other in retail shelves to attract customer attention. 

- Association rules is also known as market basket analysis.

To understand we will analyze at a set of baskets and the items in those baskets purchased by customers.

Items purchased in different baskets are:

1. Basket 1: egg, beer, sugar, bread, diaper
2. Basket 2: egg, beer, cereal, bread, diaper
3. Basket 3: milk, beer, bread
4. Basket 4: cereal, diaper, bread

<img src="baskets.png" alt="orders or baskets" style="width: 600px;"/>

- In future, if a customer buys *beer*, can we predict what he/she would most likely to buy along with *beer*. 

- To know this, we need to find out which items have shown strong association with *beer* in previous purchased baskets. We can use association rules technique to find out.

Association rule considers all possible combination of items in the previous baskets and computes various measures like 
- *support*
- *confidence*
- *lift*


The rules generated are represented as 

*{diapers} -> {beer}*, 

which means that customers who purchased diapers also purchased beer in the same basket. *{diaper, beer}* together is called itemset. 
- *{diaper}* is called the antecedent and the *{beer}* is called the consequent. 

###  Metrics

#### Support 

Support indicates the frequencies of them items appearing together with respect to all possible baskets being considered or sample size. For example, the support for (beer, diaper) will be 2/4 i.e. 50% as it appears together in 2 basket among 4 baskets being analyzed. 

So, if *X* and *Y* are items being considered, where 
- *N* is number of baskets. 
- $N_{xy}$ represents the number baskets in which *X* and *Y* appear together.
- $N_{x}$ represents the number baskets in which *X* appears.
- $N_{y}$ represents the number baskets in which *Y* appears.
   

 $supp(X,Y) = \frac{N_{xy}}{N}$
 
To filter out stronger associations, we can assume a minimum support i.e. minumum support of 0.01. This means the itemset must be present in at least one percent of baskets. It is important to use minimum support criteria to reduce the number of possible itemset 

#### Confidence

Confidence measures the proportion of the transactions that contain *X*, which also contain *Y*. *X* is called *antecedent* and *Y* is called *consequent*.

$conf(X \to Y) = \frac{N_{xy}}{N_{y}}$

#### Lift 

Lift is defined as 

$ Lift(X \to Y) = \frac{$supp(X,Y)}{$supp(X) * $supp(Y)}$


- The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. 
- Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

### Applying Association Rules

*groceries.csv* - This dataset contains transactions of a grocery store.

In [1]:
!head groceries.csv

#### Loading the dataset



In [2]:
all_txns = []

#open the file
with open('groceries.csv') as f:
    #read each line 
    content = f.readlines()
    #Remove white space from the beginning and end of the line
    txns = [x.strip() for x in content]
    # Iterate through each line and create a list of transactions
    for each_txn in txns:
        #Each transaction will contain a list of item in the transaction
        all_txns.append( each_txn.split(',') )

Print the first 5 transactions.

In [3]:
all_txns[0:5]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

#### Encoding the transactions

In [4]:
!pip install mlxtend

In [5]:
# Import all required libraries
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [6]:
# Initialize OnehotTransactions
one_hot_encoding = TransactionEncoder()
# Transform the data into one-hot-encoding format
one_hot_txns = one_hot_encoding.fit_transform(all_txns).astype("int")
# Conver the matrix into the dataframe.
one_hot_txns_df = pd.DataFrame(one_hot_txns, 
                               columns=one_hot_encoding.columns_)

Printing the first 5 transactions and items, indexed from 10 to 20.

In [7]:
one_hot_txns_df.iloc[5:10, 10:20]

,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles
5,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


It can be notices that transaction with index 5 contains an item called *butter* and transaction with index 7 contains an item *bottled beer*.

In [8]:
one_hot_txns_df.shape

(9835, 171)

The sparse matrix has a dimension of *9835 x 171*. So, a total of 9835 transactions and 171 items are available. this matrix can be fed to generate rules in the next section.

### Generating Rules

*apriori* takes the following parameters:

- df : pandas DataFrame in one-hot encoded format.
- min_support : float - A float between 0 and 1 for minumum support of the itemsets returned. Default is 0.5.
- use_colnames : bool -  If true, uses the DataFrames' column names in the returned DataFrame instead of column indices.

We will use minimum support of 0.02 i.e. the itemset is available in atleast 2 percent of all transactions.

In [9]:
frequent_itemsets = apriori(one_hot_txns_df, 
                            min_support=0.02, 
                            use_colnames=True)

Printing the 10 randomly sampled itemsets and their corresponding support.

In [10]:
frequent_itemsets.sample(10, random_state = 90)

,support,itemsets
60,0.020437,"(bottled beer, whole milk)"
52,0.033859,(sugar)
89,0.035892,"(tropical fruit, other vegetables)"
105,0.021047,"(tropical fruit, root vegetables)"
88,0.032740,"(soda, other vegetables)"
16,0.058058,(coffee)
111,0.024504,"(shopping bags, whole milk)"
36,0.079817,(newspapers)
119,0.056024,"(yogurt, whole milk)"
55,0.071683,(whipped/sour cream)


These itemsets can be passed to *association_rules* rules for generating rules and corresponding metrics.

- df :  pandas DataFrame of frequent itemsets with columns ['support', 'itemsets']
- metric : 'confidence' and 'lift'to evaluate if a rule is of interest. Default is 'confidence'.
- min_threshold : Minimal threshold for the evaluation metric to decide whether a candidate rule is of interest.

In [11]:
rules = association_rules(frequent_itemsets, # itemsets
                          metric="lift",  # lift
                          min_threshold=1)

In [12]:
rules.sample(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
52,(sausage),(other vegetables),0.093950,0.193493,0.026945,0.286797,1.482209,0.008766,1.130824
17,(whole milk),(butter),0.255516,0.055414,0.027555,0.107839,1.946053,0.013395,1.058762
93,(root vegetables),(yogurt),0.108998,0.139502,0.025826,0.236940,1.698475,0.010621,1.127694
117,(root vegetables),"(whole milk, other vegetables)",0.108998,0.074835,0.023183,0.212687,2.842082,0.015026,1.175091
46,(pork),(other vegetables),0.057651,0.193493,0.021657,0.375661,1.941476,0.010502,1.291779
43,(other vegetables),(pastry),0.193493,0.088968,0.022572,0.116658,1.311235,0.005358,1.031347
79,(rolls/buns),(sausage),0.183935,0.093950,0.030605,0.166390,1.771048,0.013324,1.086899
107,(tropical fruit),(yogurt),0.104931,0.139502,0.029283,0.279070,2.000475,0.014645,1.193594
110,(yogurt),(whipped/sour cream),0.139502,0.071683,0.020742,0.148688,2.074251,0.010742,1.090455
94,(soda),(sausage),0.174377,0.093950,0.024301,0.139359,1.483324,0.007918,1.052761


### Selecting Top 10 Rules

Let's look at top 10 rules sorted by *confidence*.

In [13]:
rules.sort_values('confidence', 
                   ascending = False)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
121,"(yogurt, other vegetables)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,0.011174,1.528340
16,(butter),(whole milk),0.055414,0.255516,0.027555,0.497248,1.946053,0.013395,1.480817
24,(curd),(whole milk),0.053279,0.255516,0.026131,0.490458,1.919481,0.012517,1.461085
115,"(other vegetables, root vegetables)",(whole milk),0.047382,0.255516,0.023183,0.489270,1.914833,0.011076,1.457687
114,"(whole milk, root vegetables)",(other vegetables),0.048907,0.193493,0.023183,0.474012,2.449770,0.013719,1.533320
28,(domestic eggs),(whole milk),0.063447,0.255516,0.029995,0.472756,1.850203,0.013783,1.412030
108,(whipped/sour cream),(whole milk),0.071683,0.255516,0.032232,0.449645,1.759754,0.013916,1.352735
91,(root vegetables),(whole milk),0.108998,0.255516,0.048907,0.448694,1.756031,0.021056,1.350401
50,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693
32,(frozen vegetables),(whole milk),0.048094,0.255516,0.020437,0.424947,1.663094,0.008149,1.294636


The probability that a customer buys *(whole mlk), given he/she has bought *(yogurt, other vegetables)* is 0.51. Now these rules can be used to create strategies like keeping the items together in store shelves or cross selling.

### Pros & Cons of Association Rules:

Advantages of using association rules are:

- Transactions data, which is used for generating rules, is always available and mostly clean.
- The rules generated are simple and can be interpreted.

- Association rules does not take the preference or ratings given by customers into account, which are important information pertaining to generating rules. 
- If customers have bought two items, but disliked one of them, then the association should not even be considered. 
- *Collaborative Filtering* leverages both, what customers bought and how they liked (rating) the items, into consideration before recommending.

## Collaborative Filtering

- Collaborative filtering is based on the notion of *similarity*. 

- For example, if two users *A* and *B* have purchased same products and have rated them similarily on a common rating scale, then *A* and *B* can be considered similar in their buying and preference behavior. 

### How to find similarity between users?

- To calculate similarity, the distance between users can be computed using the rating the users have given to the common items bought. 
- The users are similar, if the distance is small and dissimilar if the distance is large. - Most widely used metrics for distance calculations are Euclidean distance, Cosine similarity, Pearson Correlation.  

The following picture depicts three users *Bob*, *Claudia* and *Michael* and the books they have have bought and rated.

<img src="recommend_1.png" width="500">



The users are represented using their rating on the Euclidean space as below. 

<img src="recommend_2.png" width="500">


The picture shows that *Bob*'s preferences are similar to *Claudia* than *Michael*. So, the other books *Into the Wild*, which *Bob* has bought and rated high, can now be recommended to *Claudia*. 

Collaborative Filtering comes in two variations:

- User Based Similarity - Finds *K* similar users based on common items they have bought.
- Item Based Similarity - Finds *K* similar items based on common users who have bought those items.

### User Based Similarity

- *MovieLens* dataset for finding similar users based on what common movies the users have watched and how they have rated those movies. 
- The file *ratings.csv* in the dataset contain rating given by users. 
- Each line in this file represents ratings given by an user to a particular movie. The ratings are in the scale of 1 to 5.

*userId,movieId,rating,timestamp*

The dataset can be downloaded from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip.

#### Loading the dataset

Read the file using *read_csv()* method.

In [14]:
rating_df = pd.read_csv( "ml-latest-small/ratings.csv" )

Printing first 5 records.

In [15]:
rating_df.sample(10)

,userId,movieId,rating,timestamp
86455,578,6870,3.0,1418984676
89638,596,4025,3.5,1138729399
34672,247,2058,4.0,953271935
95397,626,2606,2.0,974769265
35874,259,1097,3.0,949948831
88229,587,928,4.5,1111799608
87882,585,973,5.0,975365500
45486,324,592,4.0,1451524244
20547,138,5349,0.5,1440379177
31439,228,2005,4.5,1449332941


In [16]:
rating_df.drop( 'timestamp', axis = 1, inplace = True )

Number of unique users in the dataset can be found using *unique* method on *userId* column.

In [17]:
len( rating_df.userId.unique() )

671

Similarily, number of unique movies in the dataset is,

In [18]:
len( rating_df.movieId.unique() )

9066

In [19]:
user_movies_df = rating_df.pivot( index='userId', 
                                 columns='movieId', 
                                 values = "rating" ).reset_index(drop=True)
user_movies_df.index = rating_df.userId.unique()

Print the first 5 rows and first 10 columns.

In [20]:
user_movies_df.iloc[0:5, 0:15]

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataframe contains *NaN* for those entries where users have seen a movie and not rated. We can impute those  those *NaNs* with *0* values.

In [21]:
user_movies_df.fillna( 0, inplace = True )
user_movies_df.iloc[0:5, 0:10]

movieId,1,2,3,4,5,6,7,8,9,10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Calculating Cosine Similarity between users

- Each row in *user_movies_df* represents an user. 
- So, if we compute the distance between rows, that will represent the similarity between those users. *sklearn.metrics.pairwise_distances* can be used to compute distance between all pairs of users. *pairwise_distances* takes *metric* parameter for what distance measure to use. We will be using *cosine coefficient* for finding similiarity.

Cosine similarity closer to 1 means users are very similar and closer to 0 means very dissimilar.

In [22]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation


user_sim = 1 - pairwise_distances( user_movies_df.values, metric="cosine" )
#Store the results in a dataframe
user_sim_df = pd.DataFrame( user_sim )
# set the index and column names to user ids (0 to 671)
user_sim_df.index = rating_df.userId.unique()
user_sim_df.columns = rating_df.userId.unique()

Printing the similarity between first 5 users.

In [23]:
user_sim_df.iloc[0:5, 0:5]

,1,2,3,4,5
1,1.000000,0.000000,0.000000,0.074482,0.016818
2,0.000000,1.000000,0.124295,0.118821,0.103646
3,0.000000,0.124295,1.000000,0.081640,0.151531
4,0.074482,0.118821,0.081640,1.000000,0.130649
5,0.016818,0.103646,0.151531,0.130649,1.000000


In [24]:
user_sim_df.shape

(671, 671)

In [25]:
np.fill_diagonal( user_sim, 0 )
user_sim_df.iloc[0:5, 0:5]

,1,2,3,4,5
1,0.000000,0.000000,0.000000,0.074482,0.016818
2,0.000000,0.000000,0.124295,0.118821,0.103646
3,0.000000,0.124295,0.000000,0.081640,0.151531
4,0.074482,0.118821,0.081640,0.000000,0.130649
5,0.016818,0.103646,0.151531,0.130649,0.000000


#### Filtering Similar User

To find most similar users, the maximum values of each column can be filtered. For example, most similar user to first 5 users are given by,

In [26]:
user_sim_df.idxmax(axis=1)[0:5]

1    325
2    338
3    379
4    518
5    313
dtype: int64

In [27]:
user_sim_df.iloc[1:2, 330:340]

,331,332,333,334,335,336,337,338,339,340
2,0.030344,0.002368,0.052731,0.047094,0.0,0.053044,0.05287,0.581528,0.093863,0.081814


#### 9.3.2.4 Loading the movies dataset

Movie information is contained in the file *movies.csv*. Each line of this file contains the movieid, the movie name and the movie genre.

In [28]:
movies_df = pd.read_csv( "ml-latest-small/movies.csv" )

Printing first 5 movie details.

In [29]:
movies_df[0:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


*genres* column is dropped from the dataframe, as it is not going to be used in this analysis.

In [30]:
movies_df.drop( 'genres', axis = 1, inplace = True )

#### Finding common movies of similar users

The following method takes userids of two users and returns the common movies they have watched, their ratings.

In [31]:
def get_user_similar_movies( user1, user2 ):
    # Inner join between movies watched between two users will give the common movies watched.
    common_movies = rating_df[rating_df.userId == user1].merge( 
        rating_df[rating_df.userId == user2],
        on = "movieId", 
        how = "inner" )
    # join the above result set with movies details
    return common_movies.merge( movies_df, on = 'movieId' )

Let's find out the common movies user *2* and *338* have watched together and how they have rated. We will filter movies that they have rated atleast 4.

In [32]:
common_movies = get_user_similar_movies( 2, 338 )

In [33]:
common_movies[(common_movies.rating_x >= 4.0) & 
              ((common_movies.rating_y >= 4.0))]

,userId_x,movieId,rating_x,userId_y,rating_y,title
0,2,17,5.0,338,4.0,Sense and Sensibility (1995)
2,2,47,4.0,338,4.0,Seven (a.k.a. Se7en) (1995)
5,2,150,5.0,338,4.0,Apollo 13 (1995)
28,2,508,4.0,338,4.0,Philadelphia (1993)
29,2,509,4.0,338,4.0,"Piano, The (1993)"
31,2,527,4.0,338,5.0,Schindler's List (1993)
34,2,589,5.0,338,5.0,Terminator 2: Judgment Day (1991)


#### Challenges with User similarity

- Finidng user similarity does not work for new users. 
- We need to wait until the new user buys few items and rates them. Only then users with similar preferences can be found and recommendations can be made based on that. This is called **cold start** problem in recommeder systems.

- This can be overcome by using *item based similarity*. 
- *item based similarity* is based on the notion that if two items have been bought by many users and rated similarily, then there must be some inherent relationship between these two items. 
- So in future if any user buys one of those two items, he or she will most likely buy the other one.

### Item based similarity

#### Calculating Cosine Similarity between movies

This time we need to create a pivot table, where the rows represent movies and columns represent users. And the cells in the matrix represent ratings the users has given to the movies. So, *pivot()* method will be called with *movieId* as *index* and *userId* as *columns*.

In [34]:
rating_mat = rating_df.pivot( index='movieId', 
                             columns='userId', 
                             values = "rating" ).reset_index(drop=True)
# fill all NaNs with 0
rating_mat.fillna( 0, inplace = True )
# Find the correlation between movies
movie_sim = 1 - pairwise_distances( rating_mat.values, 
                                   metric="correlation" )
# Fill the diagonal with 0, as it repreresent the auto-correlation of movies
movie_sim_df = pd.DataFrame( movie_sim )

Printing similarity between the first 5 movies.

In [35]:
movie_sim_df.iloc[0:5, 0:5]

,0,1,2,3,4
0,1.000000,0.223742,0.183266,0.071055,0.105076
1,0.223742,1.000000,0.123790,0.125014,0.193144
2,0.183266,0.123790,1.000000,0.147771,0.317911
3,0.071055,0.125014,0.147771,1.000000,0.150562
4,0.105076,0.193144,0.317911,0.150562,1.000000


In [36]:
movie_sim_df.shape

(9066, 9066)

#### Finding most similar movies

We write a method *get_similar_movies()*, which takes a *movieid* as parameter and returns the similar movies based on cosine similarity. It takes another parameter *topN* to specify how many similar movies will be returned. 

In [37]:
def get_similar_movies( movieid, topN = 5 ):
    movieidx = movies_df[movies_df.movieId == movieid].index[0]
    movies_df['similarity'] = movie_sim_df.iloc[movieidx]
    top_n = movies_df.sort_values( ["similarity"], ascending = False )[0:topN]   
    return top_n 

#### Finding similar movies to *Godfather*

Let's find out how the similarities play out, but finding out movies which are similar to the movie *Godfather*.

In [38]:
movies_df[movies_df.movieId == 858]

,movieId,title
695,858,"Godfather, The (1972)"


In [39]:
get_similar_movies(858)

,movieId,title,similarity
695,858,"Godfather, The (1972)",1.000000
977,1221,"Godfather: Part II, The (1974)",0.709246
969,1213,Goodfellas (1990),0.509372
951,1193,One Flew Over the Cuckoo's Nest (1975),0.430101
1744,2194,"Untouchables, The (1987)",0.418966


It can be observed that users who watched *'Godfather, The'*, also watched *'Godfather: Part II, The'* the most. This makes absolute sense to me! It also indicates user have watched *Goodfellas (1990)*, *One Flew Over the Cuckoo's Nest (1975)* and *Untouchables, The (1987)*.

So, in future, any user watches *'Godfather, The'*, the other movies can be recommeded to them.

#### 9.3.3.4 Finding similar movies to *Dumb & Dumber*

In [40]:
movies_df[movies_df.movieId == 231]

,movieId,title,similarity
203,231,Dumb & Dumber (Dumb and Dumber) (1994),0.054116


In [41]:
get_similar_movies(231)

,movieId,title,similarity
203,231,Dumb & Dumber (Dumb and Dumber) (1994),1.000000
309,344,Ace Ventura: Pet Detective (1994),0.635735
18,19,Ace Ventura: When Nature Calls (1995),0.509839
447,500,Mrs. Doubtfire (1993),0.485764
331,367,"Mask, The (1994)",0.461103


Most of the movies are of *comedy* genre and belonged to the actor *Jim Carry*.

## Lessons Learnt

1. How to use customer's purchase and review data for making personalized recommendations.
2. Different recommendation techniques such as association rules and collborative filtering.
3. Metrics like support, confidence and lift to filter out best association rules. 
4. Variations of Collaborative filterings techniques: user based and item based similarity.
5. Various distance metrics like Euclidean, Cosine or Pearson correlations to calculate similarity.